# Konvertering av N50 Vektor til GeoParquet

## 1. Last ned N50 Kartdata

![Geonorge N50](img/geonorge-n50.png)

Vektordata kan fritt lastes ned fra kartkatalogen.geonorge.no, du finner N50 [Her](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac)

Til denne demoen brukes:
- `Hele landet`
- `UTM 33`
- `FGBD`

![Geonorge Last ned](img/geonorge-lastned.png)

Du får en zip fil når du laster ned, pakk ut denne i `src/geoparquet`. Git er satt opp til å ikke tracke `.gdb` mapper.

In [13]:
import os
import geopandas as gpd
from pathlib import Path
from utils import get_workdir

path = get_workdir() / Path("Basisdata_0000_Norge_25833_N50Kartdata_FGDB.gdb")
n50_df = gpd.read_file(path)
n50_df.head()

/home/kluesp/projects/skygeo/src/geoparquet/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'Basisdata_0000_Norge_25833_N50Kartdata_FGDB.gdb': 'N50_Restriksjonsområder_omrade' (default), 'N50_Restriksjonsområder_grense', 'N50_Arealdekke_omrade', 'N50_Arealdekke_grense', 'N50_Samferdsel_senterlinje'. Specify layer parameter to avoid this warning.
  result = read_func(


,objtype,navn,vernedato,oppdateringsdato,verneform,allmenningtype,skytefeltstatus,SHAPE_Length,SHAPE_Area,geometry
0,Allmenning,Kjesbu,None,1000/01/01 00:00:00+00,None,Statsallmenning,None,45000.952982,4.895654e+07,"MULTIPOLYGON (((346015.34 7095731.17, 346034.1..."
1,Allmenning,Småseteråsen,None,1000/01/01 00:00:00+00,None,Statsallmenning,None,2831.752803,3.745910e+05,"MULTIPOLYGON (((341324.49 7074197.7, 341322.62..."
2,Allmenning,Limingdal statsallmenning,None,1000/01/01 00:00:00+00,None,Statsallmenning,None,9548.525436,4.152223e+06,"MULTIPOLYGON (((440651.67 7174715.21, 440674.8..."
3,Allmenning,Flottra,None,1000/01/01 00:00:00+00,None,Statsallmenning,None,632.411493,2.246830e+04,"MULTIPOLYGON (((311897.54 7177542.39, 311894.3..."
4,Allmenning,Elvdalen Rendalen statsallm,None,1000/01/01 00:00:00+00,None,Statsallmenning,None,1095.681823,4.549223e+04,"MULTIPOLYGON (((352380.59 6835971.78, 352393.6..."


In [15]:

n50_df.to_parquet("N50.snappy.parquet")